### Set working directory

In [ ]:
# Set working directory to "cims-models" location
%cd "cims-models"


### Adjust scenario parameters and optional update files below

In [2]:
# Only uncommented regions below will be run in the simulation
region_list = [
    'CIMS',
    'CAN',
    'BC',
    # 'AB',
    # 'SK',
    # 'MB',
    # 'ON',
    # 'QC',
    # 'AT',
]

# Only uncommented years below will be run in the simulation
year_list = [
    ### Historical ###
    2000,
    2005,
    2010,
    2015,
    2020,
    ### Forecast ###
    # 2025,
    # 2030,
    # 2035,
    # 2040,
    # 2045,
    # 2050,
]

# Only uncommented sectors below will be run in the simulation
# Uncomment individual sectors to calibrate one at a time
# Must use Exogenous prices (and optional exogenous demand) file below when calibrating
sector_list = [
    'Coal Mining',
    'Natural Gas Production',
    'Petroleum Crude',
    'Petroleum Refining',
    'Electricity',
    'Biodiesel',
    'Ethanol',
    'Hydrogen',
    'Mining',
    'Industrial Minerals',
    'Iron and Steel',
    'Metal Smelting',
    'Chemical Products',
    'Pulp and Paper',
    'Light Industrial',
    'Residential',
    'Commercial',
    'Transportation Personal',
    'Transportation Freight',
    'Waste',
    'Agriculture',
]



### Scenario Name
### This will be the save location for results (i.e., results_dir/scenario_name/results_general.csv)
scenario_name = 'Reference'  # Set this to current scenario (e.g., "Reference", "Net Zero")


### Base model and standard files below are required for the model to run
model_path = 'csv/model'
# Model files should be located at: cims-models/path/file/file_XX.csv

### Base model to start initialisation
base_model = 'CIMS_base'

### Default values file
default_path = 'csv/defaults/defaults_Parameters.csv'

update_files = {}


### Required sector files (included in csv/model/)
sector_req = [
    # 'sector_coal mining',
    # 'sector_natural gas extraction',
    # 'sector_petroleum crude',
    'sector_petroleum refining',
    'sector_electricity',
    'sector_biodiesel',
    'sector_ethanol',
    'sector_hydrogen',
    'sector_mining',
    'sector_industrial minerals',
    'sector_iron and steel',
    'sector_metal smelting',
    'sector_chemical products',
    'sector_pulp and paper',
    'sector_light industrial',
    'sector_residential',
    'sector_commercial',
    'sector_transportation personal',
    'sector_transportation freight',
    'sector_waste',
    'sector_agriculture',]
if sector_req:
    if model_path not in update_files:
        update_files[model_path] = []
    update_files[model_path].extend(sector_req)


### Required model files (included in csv/model/)
model_req = [
    'CIMS_DCC', # declining capital cost
    'CIMS_DIC', # declining intangible cost (neighbour effect)
    'CIMS_FIC', # fixed intangible cost; primarily used for calibration
    'CIMS_market share limits', # use of limits should be minimised
    ]
if model_req:
    if model_path not in update_files:
        update_files[model_path] = []
    update_files[model_path].extend(model_req)


### Optional model files (included in csv/model/)
model_optional = [
    'CIMS_exogenous prices',  # needed for correct calibration of historical years
    # 'CIMS_exogenous demand',  # use this file when calibrating endogenous supply sectors
    # 'CIMS_macro',
    ]
if model_optional:
    if model_path not in update_files:
        update_files[model_path] = []
    update_files[model_path].extend(model_optional)


### Temporary update files for BC EMH project (included in csv/BC_temp/)
BC_path = 'csv/BC_temp'
model_BC = [
    'sector_coal mining',  # sector structure and tech details
    'sector_natural gas production',  # sector structure and tech details
    'sector_petroleum crude',  # sector structure and tech details
    'coal mining_FIC',   # NG calibration
    'natural gas production_FIC',   # NG calibration
    'petroleum crude_FIC',   # NG calibration
    ]
if model_BC:
    if BC_path not in update_files:
        update_files[BC_path] = []
    update_files[BC_path].extend(model_BC)


### Optional scenario update files

ref_path = 'csv/policies/reference'
ref_policies = [
### Economy
    'Ref_carbon tax',
    'Ref_OBPS',
### Coal Mining
### Natural Gas Extraction
### Petroleum Crude
### Mining
### Electricity
    # 'Ref_coal phase out',
    'Ref_nuclear ban',
    # 'Ref_nuclear decommission',
### Biodiesel
### Ethanol
### Hydrogen
### Petroleum Refining
### Industrial Minerals
### Iron and Steel
### Metal Smelting
### Chemical Products
### Pulp and Paper
### Light Industrial
### Residential
    'Ref_incandescent phase out',
### Commercial
### Transportation Personal
    'Ref_LDV ZEV',
    'Ref_renewable content transport fuels',
### Transportation Freight
### Waste
    # 'Ref_waste methane large sites',
### Agriculture
    ]
if ref_policies:
    if ref_path not in update_files:
        update_files[ref_path] = []
    update_files[ref_path].extend(ref_policies)

scenario_path = 'csv/policies/net zero'    # Change this path as necessary based on current scenario
scenario_policies = [
### Economy
    # 'NZ_carbon tax',
### Coal Mining
### Natural Gas Extraction
### Petroleum Crude
### Mining
### Electricity
### Biodiesel
### Ethanol
### Hydrogen
### Petroleum Refining
### Industrial Minerals
### Iron and Steel
### Metal Smelting
### Chemical Products
### Pulp and Paper
### Light Industrial
### Residential
### Commercial
### Transportation Personal
### Transportation Freight
### Waste
### Agriculture
    ]
if scenario_policies:
    if scenario_path not in update_files:
        update_files[scenario_path] = []
    update_files[scenario_path].extend(scenario_policies)



### Model Validator

In [ ]:
### Validate model ###

# Need to reload functions in case changes were made to code
%reload_ext autoreload
%autoreload 2

import CIMS
from os import path
from datetime import datetime
import pandas as pd
import numpy as np


time_start = datetime.now()

#################### Global parameters ###########################
time_start = datetime.now()


# Leave the column list as-is to run the simulation
col_list1 = [
    'Branch',
    'Region',
    'Sector',
    'Technology',
    'Parameter',
    'Context',
    'Sub_Context',
    'Target',
    'Source',
    'Unit',
]
col_list2 = pd.Series(np.arange(2000,2051,5), dtype='string').tolist()
col_list = col_list1 + col_list2


time_start = datetime.now()

# Base model and default values files below are required for the model to run
print(f'Loading base model: {base_model}')
load_paths = []
for reg in region_list:
    load_path = f'{model_path}/{base_model}/{base_model}_{reg}.csv'
    if path.exists(load_path):
        print(f'\t{reg} - loaded')
        load_paths.append(load_path)
    else:
        print(f'\t{reg} - file does not exist')

print(f'\nLoading model updates files:')
if bool(update_files):
    update_paths = []
    for dir, file in update_files.items():
        for file in update_files[dir]:
            print(f'\n{file}')
            for reg in region_list:
                update_path = f'{dir}/{file}/{file}_{reg}.csv'
                if path.exists(update_path):
                    print(f'\t{reg} - loaded')
                    update_paths.append(update_path)
                else:
                    print(f'\t{reg} - file does not exist')

else:
    print('None')

model_validator = CIMS.ModelValidator(
            csv_file_paths = load_paths,
            col_list = col_list,
            year_list = year_list,
            sector_list = sector_list,
            scenario_files = update_paths,
            default_values_csv_path = default_path,
            )
model_validator.validate(verbose=False)

print('\nValidator: ', datetime.now() - time_start)


In [ ]:
### Show validator outputs ###

model_validator.warnings['nodes_no_provided_service']

### Run CIMS

In [ ]:
### Run model ###

# Need to reload functions in case changes were made to code
%reload_ext autoreload
%autoreload 2

import CIMS
import os
import pandas as pd
import numpy as np
from datetime import datetime


time0_start = datetime.now()


#################### Global parameters ###########################

# Leave the column list as-is to run the simulation
col_list1 = [
    'Branch',
    'Region',
    'Sector',
    'Technology',
    'Parameter',
    'Context',
    'Sub_Context',
    'Target',
    'Source',
    'Unit',
]
col_list2 = pd.Series(np.arange(2000,2051,5), dtype='string').tolist()
col_list = col_list1 + col_list2


#################### Build Model ###########################
time_start = datetime.now()

print(f'Loading base model: {base_model}')
load_paths = []
for reg in region_list:
    cur_path = f'{model_path}/{base_model}/{base_model}_{reg}.csv'
    if path.exists(cur_path):
        print(f'\t{reg} - loaded')
        load_paths.append(cur_path)
    else:
        print(f'\t{reg} - file does not exist')

model_reader = CIMS.ModelReader(
            csv_file_paths = load_paths,
            col_list = col_list,
            year_list = year_list,
            sector_list = sector_list,
            default_values_csv_path = default_path,
            )
model = CIMS.Model(model_reader)


print(f'\nLoading model updates files:')
if bool(update_files):
    update_paths = []
    for dir, file in update_files.items():
        for file in update_files[dir]:
            print(f'\n{file}')
            for reg in region_list:
                cur_path = f'{dir}/{file}/{file}_{reg}.csv'
                if path.exists(cur_path):
                    print(f'\t{reg} - loaded')
                    update_paths.append(cur_path)
                else:
                    print(f'\t{reg} - file does not exist')
            
    model_reader = CIMS.ScenarioReader(
                csv_file_paths = update_paths,
                col_list = col_list,
                year_list = year_list,
                sector_list = sector_list,
                )
    model = model.update(model_reader)
else:
    print('None')

print(f'\nModel files: ', datetime.now() - time_start)
print()


#################### Run ###########################
time_start = datetime.now()

model.run(equilibrium_threshold=0.05, max_iterations=10, show_warnings=False, print_eq=True)

print()
print('Simulation time: ', datetime.now() - time_start)

print()
print('Total time: ', datetime.now() - time0_start)



#################### Export results ###########################
results_path = f'results/{scenario_name}'
isExist = os.path.exists(results_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(results_path)

CIMS.log_model(model=model,
                output_file = f'{results_path}/results_general.csv',
                path = 'results/results_general.txt')

print()
print('Results exported')

In [ ]:
### Export tech results ###
import os

results_path = f'results/{scenario_name}'
isExist = os.path.exists(results_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(results_path)

CIMS.log_model(model=model,
                output_file = f'{results_path}/results_tech.csv',
                path = 'results/results_tech.txt')
print('\n')
print('Results exported')

In [35]:
### Print service request loops after running model ###

import pprint
pprint.pp(model.loops)